In [11]:
import torch
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          Trainer,
                          TrainingArguments,
                          BitsAndBytesConfig)

from peft import (prepare_model_for_kbit_training,
                  get_peft_model, 
                  LoraConfig, 
                  TaskType)



model_id = "meta-llama/Llama-2-13b-chat-hf"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

In [8]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False)

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_id,  quantization_config=bnb_config, use_cache=False)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.91s/it]


In [12]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [13]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                        inference_mode=False, 
                        r=64, 
                        lora_alpha=32, 
                        lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
